In [20]:
# data processing pseudocode

# read from data_algebraic directory
# parse the data, read exclusively from the support section
# copy 1 random support example and label it as the query example
# concatenate the support examples and the duplicate query example into a sentence
# tokenize each word into a number, pad it to a fixed length

# training pseudocode

# split into training and validation set -> already done
# feed the IN and OUT for each support example into the model
# feed only the IN for the query, have the model predict the OUT
# loss function to be determined ?

In [21]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim



In [22]:
# for each file in the training set of 100000, parse the file

data = []

for i in range(1, 1001):
# for i in range(1, 100001):  
    file_index = f"{i:06d}"
    file_name = file_index + ".txt"
    file_path = f"data_algebraic/train/{file_name}"
    
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
            
            # logic to parse the file
            sections = file_content.split('\n\n')
            
            support_section = None
            
            for section in sections:
                if section.startswith('*SUPPORT*'):
                    support_section = section.split('\n')[1:]  # Skip the header line
                    break 
            
            if support_section is None:
                print(f"No *SUPPORT* section found in file {file_name}.")
                continue
            
            # Convert support data to a list of tuples (IN, OUT)
            parsed_support_data = []
            for line in support_section:
                if line.startswith('IN:') and 'OUT:' in line:
                    in_part = line.split('IN:')[1].split('OUT:')[0].strip()
                    out_part = line.split('OUT:')[1].strip()
                    parsed_support_data.append((in_part, out_part))
            
            data.append(parsed_support_data)
    
    except FileNotFoundError:
        print(f"File {file_name} not found.")

# # Print the parsed data for verification
# for item in data:
#     for in_part, out_part in item:
#         print(f"IN: {in_part} OUT: {out_part}")


In [23]:
data[100]

[('dax lug', 'GREEN GREEN GREEN GREEN'),
 ('fep dax dax fep kiki zup gazzer', 'PINK YELLOW GREEN GREEN YELLOW YELLOW'),
 ('wif dax kiki', 'PURPLE GREEN GREEN'),
 ('dax dax lug', 'GREEN GREEN GREEN GREEN GREEN'),
 ('fep dax fep kiki zup wif', 'PURPLE YELLOW GREEN YELLOW YELLOW'),
 ('fep wif lug zup dax', 'GREEN YELLOW PURPLE PURPLE PURPLE PURPLE'),
 ('dax', 'GREEN'),
 ('gazzer', 'PINK'),
 ('wif lug', 'PURPLE PURPLE PURPLE PURPLE'),
 ('fep kiki', 'YELLOW YELLOW'),
 ('dax zup dax', 'GREEN GREEN'),
 ('wif kiki zup wif', 'PURPLE PURPLE PURPLE'),
 ('wif zup gazzer', 'PINK PURPLE'),
 ('wif kiki', 'PURPLE PURPLE')]

In [59]:
# tokenizer dictionary

input_list = ['SOS', 'dax', 'lug', 'wif', 'zup', 'fep', 'blicket', 'kiki', 'tufa', 'gazzer', 'EOS',]
output_list = ['SOS','RED', 'YELLOW', 'GREEN', 'BLUE', 'PURPLE', 'PINK', 'EOS']
vocab_list = ['SOS', 'dax', 'lug', 'wif', 'zup', 'fep', 'blicket', 'kiki', 'tufa', 'gazzer', 'RED', 'YELLOW', 'GREEN', 'BLUE', 'PURPLE', 'PINK',  'EOS',]

input_symbol_to_token = {symbol: float(idx + 1) for idx, symbol in enumerate(vocab_list)}
token_to_input_symbol = {float(idx + 1): symbol for idx, symbol in enumerate(vocab_list)}

# example

input_symbol = 'PINK'

input_token = input_symbol_to_token[input_symbol]

print(f"Input symbol '{input_symbol}' maps to token '{input_token}'")

# Reverse mapping example:
reverse_input_symbol = token_to_input_symbol[input_token]

print(f"Token '{input_token}' maps back to input symbol '{reverse_input_symbol}'")


Input symbol 'PINK' maps to token '16.0'
Token '16.0' maps back to input symbol 'PINK'


In [25]:
# tokenize the original data

def tokenize_sentence(sentence):
    tokens = ['SOS'] + sentence.split() + ['EOS']
    token_ids = [input_symbol_to_token[token] for token in tokens]
    return token_ids

tokenized_data = []

for sample in data:
    tokenized_sample = []
    for input_sentence, output_sentence in sample:
        input_tokens = tokenize_sentence(input_sentence)
        output_tokens = tokenize_sentence(output_sentence)
        tokenized_sample.append((input_tokens, output_tokens))
    tokenized_data.append(tokenized_sample)

In [26]:
tokenized_data[100]

[([1.0, 3.0, 4.0, 2.0], [1.0, 14.0, 14.0, 14.0, 14.0, 2.0]),
 ([1.0, 7.0, 3.0, 3.0, 7.0, 9.0, 6.0, 11.0, 2.0],
  [1.0, 17.0, 13.0, 14.0, 14.0, 13.0, 13.0, 2.0]),
 ([1.0, 5.0, 3.0, 9.0, 2.0], [1.0, 16.0, 14.0, 14.0, 2.0]),
 ([1.0, 3.0, 3.0, 4.0, 2.0], [1.0, 14.0, 14.0, 14.0, 14.0, 14.0, 2.0]),
 ([1.0, 7.0, 3.0, 7.0, 9.0, 6.0, 5.0, 2.0],
  [1.0, 16.0, 13.0, 14.0, 13.0, 13.0, 2.0]),
 ([1.0, 7.0, 5.0, 4.0, 6.0, 3.0, 2.0],
  [1.0, 14.0, 13.0, 16.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 3.0, 2.0], [1.0, 14.0, 2.0]),
 ([1.0, 11.0, 2.0], [1.0, 17.0, 2.0]),
 ([1.0, 5.0, 4.0, 2.0], [1.0, 16.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 7.0, 9.0, 2.0], [1.0, 13.0, 13.0, 2.0]),
 ([1.0, 3.0, 6.0, 3.0, 2.0], [1.0, 14.0, 14.0, 2.0]),
 ([1.0, 5.0, 9.0, 6.0, 5.0, 2.0], [1.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 5.0, 6.0, 11.0, 2.0], [1.0, 17.0, 16.0, 2.0]),
 ([1.0, 5.0, 9.0, 2.0], [1.0, 16.0, 16.0, 2.0])]

In [27]:
# Pad sequences to have uniform length
def pad_sequence(seq, max_len, padding_value=0):
    return seq + [padding_value] * (max_len - len(seq))

In [28]:
# pick one random example to treat as the query 
query_data = []

for sample in tokenized_data:
    random_sample = random.choice(sample)
    # Add the selected tuple to query_data
    query_data.append(random_sample)

# query_data

In [29]:
query_data[1]

([1.0, 5.0, 4.0, 11.0, 3.0, 2.0], [1.0, 12.0, 16.0, 16.0, 17.0, 2.0])

In [30]:
tokenized_data[1]

[([1.0, 4.0, 6.0, 2.0], [1.0, 17.0, 17.0, 2.0]),
 ([1.0, 3.0, 2.0], [1.0, 16.0, 2.0]),
 ([1.0, 8.0, 7.0, 5.0, 8.0, 2.0], [1.0, 12.0, 13.0, 13.0, 13.0, 2.0]),
 ([1.0, 8.0, 3.0, 2.0], [1.0, 13.0, 16.0, 2.0]),
 ([1.0, 5.0, 7.0, 5.0, 8.0, 2.0], [1.0, 12.0, 13.0, 12.0, 12.0, 2.0]),
 ([1.0, 8.0, 2.0], [1.0, 13.0, 2.0]),
 ([1.0, 5.0, 2.0], [1.0, 12.0, 2.0]),
 ([1.0, 3.0, 7.0, 3.0, 11.0, 8.0, 6.0, 2.0],
  [1.0, 13.0, 13.0, 13.0, 13.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 4.0, 2.0], [1.0, 17.0, 2.0]),
 ([1.0, 5.0, 11.0, 3.0, 2.0], [1.0, 16.0, 16.0, 12.0, 2.0]),
 ([1.0, 5.0, 4.0, 11.0, 3.0, 2.0], [1.0, 12.0, 16.0, 16.0, 17.0, 2.0]),
 ([1.0, 5.0, 4.0, 11.0, 5.0, 6.0, 2.0],
  [1.0, 12.0, 12.0, 12.0, 12.0, 12.0, 17.0, 2.0]),
 ([1.0, 3.0, 6.0, 2.0], [1.0, 16.0, 16.0, 2.0]),
 ([1.0, 5.0, 7.0, 4.0, 11.0, 5.0, 6.0, 2.0],
  [1.0, 12.0, 12.0, 12.0, 12.0, 17.0, 12.0, 12.0, 2.0])]

In [31]:
# Calculate the maximum input and output lengths across all sequences

max_input_len = max(len(inp) for sublist in tokenized_data for inp, _ in sublist)
max_output_len = max(len(out) for sublist in tokenized_data for _, out in sublist)

In [32]:
class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        # Return the number of sublists
        return len(self.data)

    def __getitem__(self, idx):
        
        # Get the list of tuples from the corresponding sublist
        index = self.data[idx]

        if isinstance(index, list):
            # tokenized_data is a list of a list of tuples
            # Pad each input-output pair in the sublist
            padded_data = [
                (pad_sequence(inp, max_input_len), pad_sequence(out, max_output_len))
                for inp, out in index
            ]
        else:
            # query_data is a list of tuples
            padded_data = [
                (pad_sequence(index[0], max_input_len), pad_sequence(index[1], max_output_len))
            ]

        
        # Convert each pair to a tensor
        padded_tensors = [(torch.tensor(input_padded), torch.tensor(output_padded)) for input_padded, output_padded in padded_data]

        return padded_tensors


In [45]:
# Create datasets
train_dataset = SequenceDataset(tokenized_data)
query_dataset = SequenceDataset(query_data)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
query_dataloader = DataLoader(query_dataset, batch_size=32, shuffle=False)

In [46]:
query_dataset[0][0]


(tensor([1., 4., 7., 8., 2., 0., 0., 0., 0., 0.]),
 tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [47]:
train_dataset[0]

[(tensor([1., 3., 2., 0., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 15.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 4., 7., 8., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([ 1.,  3.,  3.,  4., 11.,  2.,  0.,  0.,  0.,  0.]),
  tensor([ 1., 15., 15., 13., 15., 15., 13.,  2.,  0.,  0.])),
 (tensor([1., 9., 7., 8., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 14., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 4., 8., 2., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 2., 0., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 4., 3., 7., 9., 2., 0., 0., 0.]),
  tensor([ 1., 16., 13., 15., 14.,  2.,  0.,  0.,  0.,  0.])),
 (tensor([1., 3., 7., 9., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 15., 14.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 3., 7., 8., 2., 0., 0., 0., 0.]),
  tensor([ 1., 16.

In [48]:
def decode_sequence(tokens, token_to_symbol):
    return [token_to_symbol[token] for token in tokens if token in token_to_symbol]

In [49]:
# View a few examples from the train_dataset
for i in range(5):  # Print first 5 sublists/groups
    print(f"Group {i + 1}:")
    sublist = train_dataset[i]
    for j, (input_tensor, output_tensor) in enumerate(sublist):
        input_tokens = input_tensor.tolist()
        output_tokens = output_tensor.tolist()
        
        print(f" Example {j + 1}:")
        print(f"  Input tokens: {input_tokens}")
        print(f"  Output tokens: {output_tokens}")

        # Decode tokens to symbols for readability
        input_symbols = decode_sequence(input_tokens, token_to_input_symbol)
        output_symbols = decode_sequence(output_tokens, token_to_input_symbol)

        print(f"  Decoded Input: {input_symbols}")
        print(f"  Decoded Output: {output_symbols}")
    print("=" * 50)

Group 1:
 Example 1:
  Input tokens: [1.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'EOS']
 Example 2:
  Input tokens: [1.0, 4.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'lug', 'fep', 'blicket', 'EOS']
  Decoded Output: ['SOS', 'YELLOW', 'PURPLE', 'EOS']
 Example 3:
  Input tokens: [1.0, 3.0, 3.0, 4.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 15.0, 13.0, 15.0, 15.0, 13.0, 2.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'dax', 'lug', 'gazzer', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'BLUE', 'YELLOW', 'BLUE', 'BLUE', 'YELLOW', 'EOS']
 Example 4:
  Input tokens: [1.0, 9.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'kiki', 'fep', 'blicket', 'E

In [50]:
# View a few examples from the query dataset
for i in range(5):  # Print first 5 tuples
    print(f"Group {i + 1}:")
    input_tensor, output_tensor = query_dataset[i][0]
    input_tokens = input_tensor.tolist()
    output_tokens = output_tensor.tolist()
        
    print(f"  Input tokens: {input_tokens}")
    print(f"  Output tokens: {output_tokens}")

    # Decode tokens to symbols for readability
    input_symbols = decode_sequence(input_tokens, token_to_input_symbol)
    output_symbols = decode_sequence(output_tokens, token_to_input_symbol)

    print(f"  Decoded Input: {input_symbols}")
    print(f"  Decoded Output: {output_symbols}")
    print("=" * 50)

Group 1:
  Input tokens: [1.0, 4.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'lug', 'fep', 'blicket', 'EOS']
  Decoded Output: ['SOS', 'YELLOW', 'PURPLE', 'EOS']
Group 2:
  Input tokens: [1.0, 5.0, 4.0, 11.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 12.0, 16.0, 16.0, 17.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'wif', 'lug', 'gazzer', 'dax', 'EOS']
  Decoded Output: ['SOS', 'RED', 'PURPLE', 'PURPLE', 'PINK', 'EOS']
Group 3:
  Input tokens: [1.0, 9.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 15.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'kiki', 'dax', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'BLUE', 'BLUE', 'BLUE', 'EOS']
Group 4:
  Input tokens: [1.0, 3.0, 4.0, 5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 13.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'lug', 'wif', 'EOS']
  D

In [51]:
# output dimensions -> number of tokens 

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Forward pass through LSTM
        out, _ = self.lstm(x)
        # Output layer
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

# what is the input size in a seq2seq?

# Hyperparameters
input_size = len(input_list)
hidden_size = 50
output_size = len(output_list)
num_layers = 1
learning_rate = 0.001
num_epochs = 100

In [52]:
output_size

8

In [53]:
# Create the model
model = LSTM(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [58]:
for epoch in range (num_epochs):
    model.train()
    for batch_idx, batch in enumerate(train_dataloader):
        # Each batch is a list of tuples, so you need to extract inputs and targets
        batch_loss = 0.0  # Track the loss for each batch
        print(batch.shape())
        for seq, target in batch:
            # Ensure seq and target are float tensors
            seq = seq.float()
            if seq.dim() == 2:
                seq = seq.unsqueeze(0)
                
            target = target.float()
            if (epoch == 0 and batch_idx == 0):
                print(seq.shape)
                # print(seq)

torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])
torch.Size([1, 32, 10])


KeyboardInterrupt: 

In [55]:
# Training loop using DataLoader
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch in train_dataloader:
        # Each batch is a list of tuples, so you need to extract inputs and targets
        batch_loss = 0.0  # Track the loss for each batch
        for seq, target in batch:
            # Ensure seq and target are float tensors
            seq = seq.float()
            target = target.float()

            # Add batch dimension if needed
            if seq.dim() == 2:  # if not already batched
                seq = seq.unsqueeze(0)

            # Forward pass
            optimizer.zero_grad()
            output = model(seq)

            # Compute loss
            loss = criterion(output, target)
            batch_loss += loss.item()  # Accumulate the loss for each sample

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {batch_loss:.4f}')

# Test with a new sequence using the query_dataloader
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for test_batch in query_dataloader:
        for test_seq, _ in test_batch:  # We are only interested in inputs here
            # Ensure test_seq is a float tensor
            test_seq = test_seq.float()

            # Add batch dimension if needed
            if test_seq.dim() == 2:  # if not already batched
                test_seq = test_seq.unsqueeze(0)

            predicted_output = model(test_seq)
            print(f'Test sequence: {test_seq.squeeze().tolist()}')
            print(f'Predicted next value: {predicted_output.squeeze().item()}')


RuntimeError: input.size(-1) must be equal to input_size. Expected 11, got 10